In [1]:
from src.config import *
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('Dataset/transformed/transformed_data_6h.csv')

In [3]:
imp_lags = ['rides_t-1', 'rides_t-4', 'rides_t-28', 'rides_t-3', 'rides_t-84', 'rides_t-56', 'rides_t-24', 'rides_t-112', 'rides_t-2', 'rides_t-140','pickup_hour','target']

df_2 = df[imp_lags]

In [4]:
df_2

,rides_t-1,rides_t-4,rides_t-28,rides_t-3,rides_t-84,rides_t-56,rides_t-24,rides_t-112,rides_t-2,rides_t-140,pickup_hour,target
0,15,6,1,6,7,7,0,10,22,3,2022-01-01 00:00:00,8
1,8,6,0,22,6,0,4,8,15,8,2022-01-01 06:00:00,4
2,4,22,3,15,29,11,15,39,8,6,2022-01-01 12:00:00,4
3,4,15,3,8,24,12,11,28,4,7,2022-01-01 18:00:00,3
4,3,8,0,4,10,2,3,5,4,6,2022-01-02 00:00:00,2
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12,5,2,3,12,7,3,5,4,1,2023-12-30 18:00:00,4
8756,4,3,0,4,2,0,0,1,12,1,2023-12-31 00:00:00,2
8757,2,4,2,12,7,4,4,2,4,9,2023-12-31 06:00:00,6
8758,6,12,7,4,10,13,6,5,2,8,2023-12-31 12:00:00,6


In [5]:
# Ensure the 'pickup_hour' column is in datetime format
df_2['pickup_hour'] = pd.to_datetime(df_2['pickup_hour'])

# Split the data into training (2022) and testing (2023)
train_data = df_2[df_2['pickup_hour'].dt.year == 2022]
test_data = df_2[df_2['pickup_hour'].dt.year == 2023]

# Separate features (X) and target (Y)
X_train = train_data.drop(columns=['target','pickup_hour'])
y_train = train_data['target']

X_test = test_data.drop(columns=['target','pickup_hour'])
y_test = test_data['target']

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4380, 10)
(4380,)
(4380, 10)
(4380,)


/var/folders/wn/f9dqwf512q52fzs0t0yk_4jw0000gn/T/ipykernel_96348/343135844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['pickup_hour'] = pd.to_datetime(df_2['pickup_hour'])


In [6]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# 2. Define and train LightGBM model
model = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=100,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

model.fit(X_train, y_train)

# 3. Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"LightGBM MAE: {mae:.2f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000378 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 730
[LightGBM] [Info] Number of data points in the train set: 4380, number of used features: 10
[LightGBM] [Info] Start training from score 14.637215
LightGBM MAE: 4.70


In [7]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

import dagshub
dagshub.init(repo_owner='Vedclove', repo_name='Pedal-Predict', mlflow=True)

from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow

mlflow = set_mlflow_tracking()

import mlflow

print("Current MLflow Tracking URI:", mlflow.get_tracking_uri())

mlflow.set_experiment("LightGBM with FE Model")

# Start an MLflow run
with mlflow.start_run():
        mlflow.log_metric("mean_absolute_error", mae)

Accessing as Vedclove

Initialized MLflow to track repo "Vedclove/Pedal-Predict"

Repository Vedclove/Pedal-Predict initialized!

INFO:src.experiment_utils:MLflow tracking URI and credentials set.


Current MLflow Tracking URI: https://dagshub.com/Vedclove/Pedal-Predict.mlflow


2025/05/10 11:14:16 INFO mlflow.tracking.fluent: Experiment with name 'LightGBM with FE Model' does not exist. Creating a new experiment.


🏃 View run vaunted-fish-488 at: https://dagshub.com/Vedclove/Pedal-Predict.mlflow/#/experiments/2/runs/43585a07c62444d186b77a97747b58cf
🧪 View experiment at: https://dagshub.com/Vedclove/Pedal-Predict.mlflow/#/experiments/2
